# Creación de cargadores de datos

En esta sección se van a crear cargadores de datos de Pythorch similares a los de las seccion_02.

Anteriormente se utilizó una técnica de ventana deslizante para generar fragmentos de texto de tamaño uniforme los cuales  luego  se  agruparon  en  lotes  para  un  entrenamiento  del  modelo  más  eficiente.
Cada  fragmento  funcionó  como  una  instancia  de  entrenamiento  individual.

Sin embargo, en esta sección, se va a trabajar  con  un  conjunto  de  datos  de  spam  que  contiene  mensajes  de  texto  de  distinta  longitud.  Para  agrupar  estos  mensajes, se tienen dos opciones:

- Truncar todos los mensajes a la longitud del mensaje más corto del conjunto de datos o lote.
- Rellena todos los mensajes hasta la longitud del mensaje más largo del conjunto de datos o lote.

La  opción  1  es  computacionalmente  más  económica,  pero  puede  resultar  en  una  pérdida  significativa  de  información  si  los  mensajes  más  cortos  son  mucho  más  pequeños  que  los  mensajes  promedio  o  más  largos,  lo  que  podría  reducir  el  rendimiento  del  modelo. 

Para  implementar  la  opción  2,  donde  todos  los  mensajes  se  rellenan  hasta  la  longitud  del  mensaje  más  largo  del  conjunto  de  datos,  se añaden  tokens  de  relleno  a  todos  los  mensajes  más  cortos.  Para  ello,  se usa  "<|endoftext|>"  como  token  de  relleno.

Sin  embargo,  en  lugar  de  agregar  la  cadena  "<|endoftext|>"  a  cada  uno  de  los  mensajes  de  texto  directamente, se puede  agregar  el  ID  de  token  correspondiente  a  "<|endoftext|>"  a  los  mensajes  de  texto  codificados.

![Texto alternativo](./imgs/6.5.png)

Suponer  que  50256  es  el  ID  del  token  de  relleno  "<|endoftext|>".
Se puede verificar  que  este  sea  el  ID  de  token  correcto  codificando  "  <|endoftext|>"  usando  el  tokenizador  GPT2  del  paquete  tiktoken.

In [1]:
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")
print(tokenizer.encode("<|endoftext|>", allowed_special={"<|endoftext|>"}))

[50256]


Primero es necesario implementar  un  conjunto  de  datos  de  PyTorch,  para especificar  cómo  se  cargan  y  procesan  los  datos,  antes  de  que  se puedan  instanciar  los  cargadores  de  datos.

Para  ello,  se define  la  clase  SpamDataset ,  que  implementa  los  conceptos  ilustrados  en  la  figura anterior.  Esta  clase  SpamDataset  gestiona  varias  tareas  clave:  identifica  la  secuencia  más  larga  del  conjunto  de  datos  de  entrenamiento,  codifica  los  mensajes  de  texto  y  garantiza  que  todas  las  demás  secuencias  se  rellenen  con  un  token  de  relleno  para  que  coincida  con  la  longitud  de  la  secuencia  más  larga.secuencia.

In [2]:
import pandas as pd
import torch
from torch.utils.data import Dataset

df_train = pd.read_csv("./CSV/train.csv")
encoded_text = [tokenizer.encode(text) for text in df_train["Text"]]

maxq = 0
for text in encoded_text:
    len_1 = len(text)
    if len_1 > maxq:
        maxq = len_1

encoded_text = [text + ["50256"] * (maxq - len(text)) for text in encoded_text]

In [3]:
class SpamDataset(Dataset):
    def __init__(self, csv_file, tokenizer, max_length=None, pad_token_id=50256):
        self.data = pd.read_csv(csv_file)
        self.encoded_texts = [tokenizer.encode(text) for text in self.data["Text"]] #pre-tokenizar el texto

        if max_length is None:
            self.max_length = self._longest_encoded_length()    #truncar secuencias si son más largas que max_lenght
        else:
            self.max_length = max_length
                                                                
            self.encoded_texts = [  
                encoded_text[:self.max_length]
                for encoded_text in self.encoded_texts
            ]

        self.encoded_texts = [text + [pad_token_id] * (maxq - len(text)) for text in self.encoded_texts]
    
    def __getitem__(self, index):
        encoded = self.encoded_texts[index]
        label = self.data.loc[index]["Label"]
        return (torch.tensor(encoded), torch.tensor(label))

    def __len__(self):
        return len(self.data)

    def _longest_encoded_length(self):
        maxq = 0
        for text in self.encoded_texts:
            len_1 = len(text)
            if len_1 > maxq:
                maxq = len_1
        return maxq

La  clase  SpamDataset  carga  datos  de  los  archivos  CSV,  tokeniza  el  texto  mediante  el  tokenizador  GPT2  de  tiktoken  y  permite  rellenar  o  truncar  las  secuencias  a  una  longitud  uniforme,  determinada  por  la  secuencia  más  larga  o  una  longitud  máxima  predefinida.  Esto  garantiza  que  cada  tensor  de  entrada  tenga  el  mismo  tamaño,  lo  cual  es  necesario  para  crear  los  lotes  en  el  cargador  de  datos  de  entrenamiento:

In [4]:
train_dataset = SpamDataset(
    csv_file="CSV/train.csv",
    max_length=None,
    tokenizer=tokenizer
)

A taner en cuenta que  la  longitud  de  secuencia  más  larga  se  almacena  en  el  atributo  max_length  del  conjunto  de  datos .

In [5]:
print(train_dataset.max_length)

120


El  código  genera  120,  lo  que  indica  que  la  secuencia  más  larga  no  contiene  más  de  120  tokens,  una  longitud  común  para  mensajes  de  texto.  Cabe  destacar  que  el  modelo  admite  secuencias  de  hasta  1024  tokens,  dado  el  límite  de  longitud  de  contexto.  Si  su  conjunto  de  datos  incluye  textos  más  largos,  puede  pasar  max_length=1024  al  crear  el  conjunto  de  datos  de  entrenamiento  en  el  código  anterior  para  garantizar  que  los  datos  no  excedan  la  longitud  de  entrada  (contexto)  admitida  por  el  modelo.

A continuación, se rellena los conjuntos de validación y prueba para que coincidan con la longitud de la secuencia de entrenamiento más larga. Es importante tener en cuenta que cualquier muestra del conjunto de validación y prueba que exceda la longitud máxima de entrenamiento se trunca mediante encoded_text[:self.max_length] en el código de SpamDataset que que definio anteriormente.

El truncamiento es opcional; también puede establecer max_length=None tanto para los conjuntos de validación como de prueba, siempre que no haya secuencias que excedan los 1.024 tokens en estos conjuntos.

In [6]:
val_dataset = SpamDataset(
    csv_file="CSV/validation.csv",
    max_length=train_dataset.max_length,
    tokenizer=tokenizer
)
test_dataset = SpamDataset(
    csv_file="CSV/test.csv",
    max_length=train_dataset.max_length,
    tokenizer=tokenizer
)

Usando  los  conjuntos  de  datos  como  entradas,  se pueden  instanciar  los  cargadores  de  datos.  Sin  embargo,  en  este  caso,  los  objetivos  representan  etiquetas  de  clase  en  lugar  de  la  siguiente tokens  en  el  texto.  Por  ejemplo,  al  elegir  un  tamaño  de  lote  de  8,  cada  lote  constará  de  ejemplos  de  entrenamiento  de  longitud  120  y  la  etiqueta  de  clase  correspondiente  de  cada  ejemplo.

![Texto alternativo](./imgs/6.6.png)

El  siguiente  código  se crea  los  cargadores  de  datos  del  conjunto  de  entrenamiento,  validación  y  prueba  que  cargan  los  mensajes  de  texto  y  las  etiquetas  en  lotes  de  tamaño  8.


In [7]:
from torch.utils.data import DataLoader

num_workers = 0
batch_size = 8
torch.manual_seed(123)
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    drop_last=True,
)
val_loader = DataLoader(
    dataset=val_dataset,
    batch_size=batch_size,
    num_workers=num_workers,
    drop_last=False,
)
test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    num_workers=num_workers,
    drop_last=False,
)

In [8]:
#Comprobación de que los cargadores están funcionando
for input_batch, target_batch in train_loader:
    pass
print("Input batch dimensions:", input_batch.shape)
print("Label batch dimensions", target_batch.shape)

Input batch dimensions: torch.Size([8, 120])
Label batch dimensions torch.Size([8])


Como se puede ver, los lotes de entrada constan de 8 ejemplos de entrenamiento con  120  tokens  cada  uno,  como  se  esperaba.  El  tensor  de  etiquetas  almacena  las  etiquetas  de  clase  correspondientes  a  los  8  ejemplos  de  entrenamiento. Por  último,  para  tener  una  idea  del  tamaño  del  conjunto  de  datos,  se imprimen el  número  total  de  lotes  en  cada  conjunto  de  datos.

In [9]:
print(f"{len(train_loader)} training batches")
print(f"{len(val_loader)} validation batches")
print(f"{len(test_loader)} test batches")

130 training batches
19 validation batches
38 test batches


Con  esto  concluye  la  preparación  de  datos  de  este  capítulo.  A  continuación, se preparará el  modelo  para  el  ajuste  fino.

[Inicialización de un modelo con pesos preentrenados](./4_inicializacion_modelo_pesos_preentrenados.ipynb)